In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [101]:
url = "https://www.sec.gov/Archives/edgar/data/1675654/000089109216015335/e00525ex99-5.htm"

headers_sec = {'User-Agent': 'The Oakleaf Group - jake.heid@theoakleafgroup.com - (A RMBS consulting company)',}

html = requests.get(url, headers = headers_sec).content

soup = BeautifulSoup(html, 'html.parser')
table = soup.find("table")

df_exception = pd.read_html(str(table))[0]

df_exception.rename(columns=df_exception.iloc[0], inplace=True)
df_exception = df_exception.iloc[1:]


In [102]:
url = "https://www.sec.gov/Archives/edgar/data/1675654/000089109216015335/e00525ex99-3.htm"

headers_sec = {'User-Agent': 'The Oakleaf Group - jake.heid@theoakleafgroup.com - (A RMBS consulting company)',}

html = requests.get(url, headers = headers_sec).content

soup = BeautifulSoup(html, 'html.parser')
table = soup.find("table")

df_integrity = pd.read_html(str(table))[0]

#df_integrity.rename(columns=df_integrity.iloc[0], inplace=True)
#df_integrity = df_integrity.iloc[1:]

df_integrity.rename(columns=df_integrity.iloc[3], inplace=True)
df_integrity = df_integrity.iloc[4:]



In [103]:
url = "https://www.sec.gov/Archives/edgar/data/1675654/000089109216015335/e00525ex99-2.htm"

headers_sec = {'User-Agent': 'The Oakleaf Group - jake.heid@theoakleafgroup.com - (A RMBS consulting company)',}

html = requests.get(url, headers = headers_sec).content

soup = BeautifulSoup(html, 'html.parser')
table = soup.find("table")

df_loan_level_rating = pd.read_html(str(table))[0]

df_loan_level_rating.rename(columns=df_loan_level_rating.iloc[0], inplace=True)
df_loan_level_rating = df_loan_level_rating.iloc[1:]



In [104]:
url = 'https://www.sec.gov/Archives/edgar/data/1675654/000089109216015335/e00525ex99-4.htm'

headers_sec = {'User-Agent': 'The Oakleaf Group - jake.heid@theoakleafgroup.com - (A RMBS consulting company)',}

html = requests.get(url, headers = headers_sec).content

soup = BeautifulSoup(html, 'html.parser')
table = soup.find("table")

df_valuation = pd.read_html(str(table))[0]

df_valuation.rename(columns=df_valuation.iloc[0], inplace=True)
df_valuation = df_valuation.iloc[1:]

new_columns = []
for x in range(len(df_valuation.columns)):
    new_columns.append((df_valuation.columns[x], df_valuation.iloc[0][x]))

df_valuation.columns = pd.MultiIndex.from_tuples(new_columns)
df_valuation = df_valuation.iloc[1:]

#reset index
df_valuation = df_valuation.reset_index(drop=True)


In [111]:
df_valuation_specific_columns = df_valuation[[('LOAN INFORMATION', 'AMC Loan ID'), ('ORIGINATION VALUES', 'Appraised Value'),('ORIGINATION VALUES', 'Appraisal Date')]]
#make the subcolumns the columns
df_valuation_specific_columns.columns = df_valuation_specific_columns.columns.droplevel(0)
df_valuation_specific_columns

,AMC Loan ID,Appraised Value,Appraisal Date
0,200452262,"$140,000.00",11/24/2015
1,200343486,"$403,000.00",10/27/2015
2,200419913,"$900,000.00",12/21/2015
3,200459830,"$313,000.00",11/30/2015
4,200391306,"$71,000.00",10/19/2015
...,...,...,...
363,200419922,"$509,000.00",01/15/2016
364,200419928,"$425,000.00",01/06/2016
365,200393961,"$980,000.00",11/20/2015
366,200466720,"$1,050,000.00",02/08/2016


In [106]:
cols_to_use = df_exception.columns.difference(df_loan_level_rating.columns)
#add AMC Loan ID to cols_to_use
cols_to_use = cols_to_use.append(pd.Index(['Loan Exception ID']))

#merge df_exception and df_loan_level_rating on Loan Exception ID
df_merged = pd.merge(df_loan_level_rating, df_exception[cols_to_use], on='Loan Exception ID', how='outer')

df_merged = pd.merge(df_merged, df_valuation_specific_columns, on='AMC Loan ID', how='outer')

#make loan exception ID the lindex
df_merged = df_merged.set_index('Loan Exception ID')
#make the index the first column
df_merged = df_merged.reset_index()
#sort by loan ID
df_merged = df_merged.sort_values(by='AMC Loan ID')

In [107]:
df_merged_credit = df_merged[df_merged['Exception Type'] == 'Credit']


In [108]:
columns_to_keep = ['Loan Exception ID', 'AMC Loan ID', 'Exception Date', 'Exception Type', 'Exception Category', 'Exception Subcategory', 'Exception',
'Property State', 'Occupancy', 'Purpose', 'Fitch Overall Final', 'Fitch Credit Final', 'Fitch Compliance Final', 'Fitch Property Final', 'DBRS Final Exception Rating',
'Fitch Final Exception Rating', 'Originator QM ATR Status']

In [109]:
df_merged_credit_specific_columns = df_merged_credit[columns_to_keep]

In [110]:
writer = pd.ExcelWriter('COLT 2016-1 Merged Data_v3.xlsx', engine='xlsxwriter')

df_merged.to_excel(writer, sheet_name='Rating + Exception')
df_merged_credit.to_excel(writer, sheet_name='Rating + Exception Credit')
df_merged_credit_specific_columns.to_excel(writer, sheet_name='Specific Columns')
df_integrity.to_excel(writer, sheet_name='Integrity')

writer.save()
